<a href="https://colab.research.google.com/github/niladridutta1988/wf_agentic_ai_training/blob/main/Batch2/day4_mcp/mcp_testing_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MCP (Model Context Protocol) Testing with LangChain

This notebook demonstrates how to:
1. Set up MCP servers (Math and Weather)
2. Connect to them using different transports (stdio and HTTP)
3. Use them with LangChain agents

**Author:** Created for testing MCP integration

**Date:** January 2026

## Step 1: Install Dependencies

Install all required packages for MCP and LangChain

In [1]:
print("📦 Installing dependencies...")
!pip install -q fastmcp mcp langchain langchain-groq langchain-mcp-adapters python-dotenv
print("✅ Installation complete!")

📦 Installing dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 816.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.2/416.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.8/197.8 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.4/96.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.3/96.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.6/119.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.2/354.2 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

## Step 2: Set Up API Key

**Option 1 (Recommended):** Use Colab Secrets
- Click the 🔑 key icon in the left sidebar
- Add new secret: Name = `GROQ_API_KEY`, Value = your API key
- Enable "Notebook access"

**Option 2:** Enter manually when prompted

In [2]:
import os
from google.colab import userdata

try:
    os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    print("⚠️  No secret found. Enter your API key manually:")
    groq_api_key = input("Enter GROQ_API_KEY: ")
    os.environ['GROQ_API_KEY'] = groq_api_key
    print("✅ API key set")

✅ API key loaded from Colab secrets


## Step 3: Create Math Server

This server provides `add` and `multiply` tools via stdio transport

In [3]:
%%writefile math_server.py
from fastmcp import FastMCP
import sys

# Ensure unbuffered output for stdio transport
sys.stdout.reconfigure(line_buffering=True)
sys.stderr.reconfigure(line_buffering=True)

mcp = FastMCP("Math")

@mcp.tool()
def add(a: int, b: int) -> int:
    """Add two numbers"""
    print(f"[Math Server] Adding {a} + {b}", file=sys.stderr)
    return a + b

@mcp.tool()
def multiply(a: int, b: int) -> int:
    """Multiply two numbers"""
    print(f"[Math Server] Multiplying {a} * {b}", file=sys.stderr)
    return a * b

if __name__ == "__main__":
    print("[Math Server] Starting...", file=sys.stderr)
    mcp.run(transport="stdio")

Writing math_server.py


## Step 4: Create Weather Server

This server provides `get_weather` tool via HTTP transport

In [4]:
%%writefile weather_server.py
from fastmcp import FastMCP

mcp = FastMCP("Weather")

@mcp.tool()
async def get_weather(location: str) -> str:
    """Get weather for location."""
    return f"It's always sunny in {location}!"

if __name__ == "__main__":
    print("Starting weather server on port 8000...")
    mcp.run(transport="streamable-http", port=8000)

Writing weather_server.py


## Step 5: Start Weather Server in Background

The weather server runs as an HTTP endpoint

In [5]:
import subprocess
import time
import requests

# Kill any existing server on port 8000
!pkill -f weather_server.py 2>/dev/null || true
time.sleep(1)

# Start weather server in background
print("🌤️  Starting weather server...")
weather_process = subprocess.Popen(
    ['python', 'weather_server.py'],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# Wait for server to start
print("⏳ Waiting for server to start...")
time.sleep(3)

# Test if server is running
try:
    response = requests.get('http://localhost:8000/mcp')
    print(f"✅ Weather server is running! Status: {response.status_code}")
except Exception as e:
    print(f"⚠️  Weather server check: {e}")

^C
🌤️  Starting weather server...
⏳ Waiting for server to start...
⚠️  Weather server check: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /mcp (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f490742db20>: Failed to establish a new connection: [Errno 111] Connection refused'))


## Step 6: Test Math Server Standalone

Test the math server directly using MCP protocol

In [6]:
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client

async def test_math_server():
    print("🧮 Testing Math Server...")
    print("-" * 60)

    try:
        server_params = StdioServerParameters(
            command="python",
            args=["-u", "math_server.py"],
        )

        read_stream, write_stream = await stdio_client(server_params)

        async with ClientSession(read_stream, write_stream) as session:
            await session.initialize()
            print("✓ Server initialized")

            # List tools
            tools_result = await session.list_tools()
            print(f"✓ Found {len(tools_result.tools)} tools:")
            for tool in tools_result.tools:
                print(f"  - {tool.name}: {tool.description}")

            # Test add
            add_result = await session.call_tool("add", {"a": 3, "b": 5})
            print(f"\n✓ add(3, 5) = {add_result.content[0].text}")

            # Test multiply
            multiply_result = await session.call_tool("multiply", {"a": 4, "b": 6})
            print(f"✓ multiply(4, 6) = {multiply_result.content[0].text}")

            print("\n✅ Math server test passed!")
            return True

    except Exception as e:
        print(f"\n❌ Math server test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

# Run the test
await test_math_server()

🧮 Testing Math Server...
------------------------------------------------------------

❌ Math server test failed: object _AsyncGeneratorContextManager can't be used in 'await' expression


Traceback (most recent call last):
  File "/tmp/ipython-input-2632049504.py", line 15, in test_math_server
    read_stream, write_stream = await stdio_client(server_params)
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: object _AsyncGeneratorContextManager can't be used in 'await' expression


False

## Step 7: Test Weather Server Standalone

Test the weather server directly using MCP protocol

In [7]:
from mcp import ClientSession
from mcp.client.sse import sse_client
import uuid

async def test_weather_server():
    print("🌤️  Testing Weather Server...")
    print("-" * 60)

    try:
        session_id = str(uuid.uuid4())

        async with sse_client(
            url="http://localhost:8000/mcp",
            headers={"mcp-session-id": session_id}
        ) as (read_stream, write_stream):

            async with ClientSession(read_stream, write_stream) as session:
                await session.initialize()
                print("✓ Server initialized")

                # List tools
                tools_result = await session.list_tools()
                print(f"✓ Found {len(tools_result.tools)} tools:")
                for tool in tools_result.tools:
                    print(f"  - {tool.name}: {tool.description}")

                # Test weather
                weather_result = await session.call_tool("get_weather", {"location": "New York"})
                print(f"\n✓ get_weather('New York') = {weather_result.content[0].text}")

                print("\n✅ Weather server test passed!")
                return True

    except Exception as e:
        print(f"\n❌ Weather server test failed: {e}")
        import traceback
        traceback.print_exc()
        return False

# Run the test
await test_weather_server()

🌤️  Testing Weather Server...
------------------------------------------------------------

❌ Weather server test failed: unhandled errors in a TaskGroup (1 sub-exception)


  + Exception Group Traceback (most recent call last):
  |   File "/tmp/ipython-input-3085164297.py", line 12, in test_weather_server
  |     async with sse_client(
  |                ^^^^^^^^^^^
  |   File "/usr/lib/python3.12/contextlib.py", line 210, in __aenter__
  |     return await anext(self.gen)
  |            ^^^^^^^^^^^^^^^^^^^^^
  |   File "/usr/local/lib/python3.12/dist-packages/mcp/client/sse.py", line 63, in sse_client
  |     async with anyio.create_task_group() as tg:
  |                ^^^^^^^^^^^^^^^^^^^^^^^^^
  |   File "/usr/local/lib/python3.12/dist-packages/anyio/_backends/_asyncio.py", line 783, in __aexit__
  |     raise BaseExceptionGroup(
  | ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Traceback (most recent call last):
    |   File "/usr/local/lib/python3.12/dist-packages/httpx/_transports/default.py", line 101, in map_httpcore_exceptions
    |     yield
    |   File "/usr/local/lib/python3.

False

## Step 8: Run Full MCP Client with LangChain Agent

This combines both servers with a LangChain agent that can use all tools

In [8]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain.agents import create_agent
from langchain_groq import ChatGroq
import sys

async def run_mcp_client():
    print("=" * 70)
    print("🚀 MCP CLIENT WITH LANGCHAIN AGENT")
    print("=" * 70)

    # Configure servers
    servers = {
        "math": {
            "transport": "stdio",
            "command": sys.executable,
            "args": ["-u", "math_server.py"],
        },
        "weather": {
            "transport": "streamable_http",
            "url": "http://localhost:8000/mcp",
        },
    }

    print("\n📋 Creating MCP client...")
    client = MultiServerMCPClient(servers)

    try:
        print("🔧 Getting tools...")
        tools = await asyncio.wait_for(client.get_tools(), timeout=15)
        print(f"✓ Got {len(tools)} tools:")
        for tool in tools:
            print(f"  - {tool.name}")
    except Exception as e:
        print(f"❌ Failed to get tools: {e}")
        import traceback
        traceback.print_exc()
        return

    try:
        print("\n🤖 Creating agent...")
        llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)
        agent = create_agent(llm, tools)
        print("✓ Agent created")
    except Exception as e:
        print(f"❌ Failed to create agent: {e}")
        import traceback
        traceback.print_exc()
        return

    # Test 1: Math query
    print("\n" + "=" * 70)
    print("🧮 TEST 1: Math Query")
    print("=" * 70)
    print("Question: What's (3 + 5) x 12?")

    try:
        math_response = await asyncio.wait_for(
            agent.ainvoke({
                "messages": [{"role": "user", "content": "what's (3 + 5) x 12?"}]
            }),
            timeout=30,
        )
        print("\n📊 Answer:")
        print(math_response['messages'][-1].content)
    except Exception as e:
        print(f"❌ Math query failed: {e}")
        import traceback
        traceback.print_exc()

    # Test 2: Weather query
    print("\n" + "=" * 70)
    print("🌤️  TEST 2: Weather Query")
    print("=" * 70)
    print("Question: What is the weather in NYC?")

    try:
        weather_response = await asyncio.wait_for(
            agent.ainvoke({
                "messages": [{"role": "user", "content": "what is the weather in nyc?"}]
            }),
            timeout=30,
        )
        print("\n📊 Answer:")
        print(weather_response['messages'][-1].content)
    except Exception as e:
        print(f"❌ Weather query failed: {e}")
        import traceback
        traceback.print_exc()

    # Test 3: Combined query
    print("\n" + "=" * 70)
    print("🔀 TEST 3: Combined Query")
    print("=" * 70)
    print("Question: What is 7 times 8, and what's the weather in Paris?")

    try:
        combined_response = await asyncio.wait_for(
            agent.ainvoke({
                "messages": [{"role": "user", "content": "What is 7 times 8, and what's the weather in Paris?"}]
            }),
            timeout=30,
        )
        print("\n📊 Answer:")
        print(combined_response['messages'][-1].content)
    except Exception as e:
        print(f"❌ Combined query failed: {e}")
        import traceback
        traceback.print_exc()

    print("\n" + "=" * 70)
    print("✅ ALL TESTS COMPLETE!")
    print("=" * 70)

# Run the full client
await run_mcp_client()

🚀 MCP CLIENT WITH LANGCHAIN AGENT

📋 Creating MCP client...
🔧 Getting tools...
❌ Failed to get tools: fileno


Traceback (most recent call last):
  File "/tmp/ipython-input-241423002.py", line 29, in run_mcp_client
    tools = await asyncio.wait_for(client.get_tools(), timeout=15)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/asyncio/tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_mcp_adapters/client.py", line 197, in get_tools
    tools_list = await asyncio.gather(*load_mcp_tool_tasks)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_mcp_adapters/tools.py", line 478, in load_mcp_tools
    async with create_session(
               ^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/contextlib.py", line 210, in __aenter__
    return await anext(self.gen)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/langchain_mcp_adapters/sessions.py", line 419, in create_session
    as

## Step 9: Cleanup (Optional)

Stop the weather server when done

In [9]:
print("🧹 Cleaning up...")
!pkill -f weather_server.py
print("✅ Weather server stopped")

🧹 Cleaning up...
✅ Weather server stopped
